In [1]:
import argparse
import os
import time
import datetime
import torch
import torch.optim as optim
from math import ceil
from torch.utils.data import DataLoader
from dataset.dataset import CityScapesDataset
from utils.metric import runningScore, averageMeter
import numpy as np
from tqdm import tqdm
import torch.nn as nn



os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
torch.cuda.is_available()

True

In [3]:
# n_train=2436
n_train=2476
n_val=400


batchsize   = 16
Epoch       = 25
img_size    = [256, 512]
model_name  = 'model_task1'
task        = 'cat'
lr = 2e-3

final_save_path='./'+model_name+'.pth'

In [4]:
def train(epoch, data_loader, Net, optimizer, loss_fn, Meter):
    Net.train()
    timeStart = time.time()
    with tqdm(total=n_train, desc=f'Epoch {epoch + 1}/{Epoch}', unit='img') as pbar:
        for i, (data, target) in enumerate(data_loader):
            data , target = data.to(device),target.to(device)
            ##yourself
            optimizer.zero_grad()
            pred = Net(data)
            loss = loss_fn(pred, target)
            loss.backward()
            optimizer.step()
            
            training_loss = loss.item()
            pbar.set_postfix(**{'loss (batch)': training_loss})
            pred = pred.data.max(1)[1]
            Meter['metric'].update(target.data.cpu().numpy(), pred.data.cpu().numpy())
            Meter['loss'].update(training_loss,data.size()[0])
            pbar.update(data.shape[0])
    timeEnd = time.time()
    score, class_iou = Meter['metric'].get_scores()
    loss_avg = Meter['loss'].avg
    print('epoch %3d : %10s loss: %f OverallAcc: %f MeanAcc %f mIoU %f time: %f' 
        %(epoch, ('training'), loss_avg, score['OverallAcc'], score['MeanAcc'], score['mIoU'], timeEnd-timeStart))

    return score['mIoU'],loss_avg

In [5]:
def val(epoch, data_loader, Net, loss_fn, Meter):
    Net.eval()
    with torch.no_grad():
        for i, (data, target) in enumerate(data_loader):
            data, target = data.to(device), target.to(device)
            timeStart = time.time()
            ### evaluate by yourself
            pred = Net(data)
            validation_loss = loss_fn(pred, target).item()
            timeEnd = time.time()           
            
            pred = pred.data.max(1)[1]
            Meter['metric'].update(target.data.cpu().numpy(), pred.data.cpu().numpy())
            Meter['loss'].update(validation_loss,data.size()[0])
            Meter['time'].update(timeEnd-timeStart,1)
    score, class_iou = Meter['metric'].get_scores()
    loss_avg = Meter['loss'].avg
    time_avg = Meter['time'].avg
    print('epoch %3d : %10s loss: %f OverallAcc: %f MeanAcc %f mIoU %f time: %f' 
        %(epoch, ('validation'), loss_avg, score['OverallAcc'], score['MeanAcc'], score['mIoU'], time_avg))
    
    return score['mIoU']

In [6]:
best_val_miou=0
current_val_miou=0

In [7]:
import network
#from network import *
if __name__ == '__main__':

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")   
    assert task in ['cat'],'wrong value of task'
    if task=='cat':
        num_classes = 8
        
    training_meter = {'metric':runningScore(num_classes),'loss':averageMeter(),'time':averageMeter()}
    validation_meter = {'metric':runningScore(num_classes),'loss':averageMeter(),'time':averageMeter()}

    print(str(datetime.datetime.now()))
    print('batchsize %3d | epoch %3d | img_size  %4d %4d | task  %6s | model_name  %25s '
                    %(batchsize,Epoch,img_size[0],img_size[1], task ,model_name ))

    TrainingDataset   = CityScapesDataset("/export/home/dl2025f/shared/data", "training", img_size, task=task, augmentation=None)
    ValidationDataset = CityScapesDataset("/export/home/dl2025f/shared/data", "validation", img_size, task=task)

    TrainingLoader    = DataLoader(TrainingDataset, batch_size=batchsize, shuffle=True, num_workers=4)
    ValidationLoader  = DataLoader(ValidationDataset, batch_size=batchsize, shuffle=False, num_workers=4)
    num_batch         = ceil(len(TrainingDataset)/batchsize)

    # define yout model
    Net = network.UNet(n_channels=3, n_classes=num_classes)
    Net = Net.to(device)
    # define your optimizer
    optimizer = optim.Adam(Net.parameters(), lr=lr)
    # scheduler
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
    # define your loss   
    loss_fn = nn.CrossEntropyLoss()

    start_epoch = 0
       
    for epoch in range(start_epoch, Epoch):
        for _, v in training_meter.items():
            v.reset()
            
        show_lr=optimizer.param_groups[0]['lr']
        print('learning rate is : ',  show_lr)
        current_train_miou,current_train_loss=train(epoch, TrainingLoader, Net, optimizer, loss_fn, training_meter)
        
        scheduler.step() ###  StepLR
        
        if (epoch+1)%1==0 or epoch==Epoch-1:
            for _, v in validation_meter.items():
                v.reset()
            current_val_miou=val(epoch, ValidationLoader, Net, loss_fn, validation_meter)
            
            if current_val_miou>best_val_miou:
                best_val_miou=current_val_miou
                best_state_dict = Net.state_dict()
                torch.save(best_state_dict, final_save_path)
                print("(model saved)")
    print(str(datetime.datetime.now()))

2025-11-27 16:32:33.107659
batchsize  16 | epoch  25 | img_size   256  512 | task     cat | model_name                model_task1 
learning rate is :  0.002


Epoch 1/25: 100%|██████████| 2476/2476 [01:59<00:00, 20.74img/s, loss (batch)=0.473]

epoch   0 :   training loss: 0.729188 OverallAcc: 0.767928 MeanAcc 0.544855 mIoU 0.434559 time: 119.360943


epoch   0 : validation loss: 0.578124 OverallAcc: 0.817678 MeanAcc 0.612556 mIoU 0.500657 time: 0.124723
(model saved)
learning rate is :  0.002


Epoch 2/25: 100%|██████████| 2476/2476 [01:54<00:00, 21.61img/s, loss (batch)=0.44] 

...

epoch  23 :   training loss: 0.194690 OverallAcc: 0.934036 MeanAcc 0.854269 mIoU 0.781661 time: 115.974708


epoch  23 : validation loss: 0.278457 OverallAcc: 0.912547 MeanAcc 0.834211 mIoU 0.751199 time: 0.124803
(model saved)
learning rate is :  0.001


Epoch 25/25: 100%|██████████| 2476/2476 [01:55<00:00, 21.41img/s, loss (batch)=0.195]

epoch  24 :   training loss: 0.194590 OverallAcc: 0.933573 MeanAcc 0.854462 mIoU 0.780516 time: 115.659122


epoch  24 : validation loss: 0.263163 OverallAcc: 0.916547 MeanAcc 0.846189 mIoU 0.760689 time: 0.124830
(model saved)
2025-11-27 17:30:15.128288


In [8]:
print(' best_val_miou is :  ',best_val_miou)

 best_val_miou is :   0.7606891829551075
